In [1]:
import tensorflow as tf

c = tf.constant("Hello distributed tensorflow !")
server = tf.train.Server.create_local_server()

In [2]:
with tf.Session(server.target) as sess:
    print(sess.run(c))

b'Hello distributed tensorflow !'


In [3]:
server.target

'grpc://localhost:52637'

In [22]:
# simple placement

with tf.device("/cpu:0"):
    a = tf.Variable(3.0)
    b = tf.Variable(4.0)

c = a * b

# cpu:0 aggregates all cpu on a multi-CPU system
# currently there is no way to pin nodes on specific CPUs
# or to use a subset of all cpu

In [7]:
# Loggin placements

config = tf.ConfigProto()
config.log_device_placement = True
sess = tf.Session(config=config)

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce GTX 1050, pci bus id: 0000:01:00.0, compute capability: 6.1



In [28]:
init = tf.global_variables_initializer()

In [17]:
i = tf.global_variables()
print(i)

[<tf.Variable 'Variable:0' shape=() dtype=float32_ref>]


In [24]:
with tf.Session() as sess:
    a.initializer.run()
    b.initializer.run()
    result = c.eval()

print(result)

12.0


In [36]:
# with using our config variable

with tf.Session(config=config) as sess:
    with tf.device("/cpu:0"):
        init.run()
        result = c.eval()
        print(result)

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce GTX 1050, pci bus id: 0000:01:00.0, compute capability: 6.1

12.0


In [40]:
import subprocess

n = str(subprocess.check_output(["nvidia-smi", "-L"]))
print(n)

b'GPU 0: GeForce GTX 1050 (UUID: GPU-b0ade927-3fa3-0bfa-0d06-c1a5174b532d)\r\n'


In [48]:
# Dynamic placment

def variables_on_cpu(op):
    if op.type == "Variable":
        return "/cpu:0"
    else:
        return "/gpu:0"

with tf.device(variables_on_cpu):
    a = tf.Variable(3.0)
    b = tf.constant(4.0)
    # b = tf.Variable(4.0)
    
c = a * b

with tf.Session(config=config) as sess:
    init.run()
    try:
        result = c.eval()
        print(result)
    except Exeption as e:
        print(e)
    #throwing error for some reason

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce GTX 1050, pci bus id: 0000:01:00.0, compute capability: 6.1



InvalidArgumentError: Cannot assign a device for operation Variable_7: Could not satisfy explicit device specification '/device:GPU:0' because no supported kernel for GPU devices is available.
Colocation Debug Info:
Colocation group had the following types and supported devices: 
Root Member(assigned_device_name_index_=-1 requested_device_name_='/device:GPU:0' assigned_device_name_='' resource_device_name_='/device:GPU:0' supported_device_types_=[CPU] possible_devices_=[]
VariableV2: CPU 
Assign: CPU 
Identity: GPU CPU 

Colocation members, user-requested devices, and framework assigned devices, if any:
  Variable_7 (VariableV2) /device:GPU:0
  Variable_7/Assign (Assign) /device:GPU:0
  Variable_7/read (Identity) /device:GPU:0

Op: VariableV2
Node attrs: shape=[], shared_name="", dtype=DT_INT32, container=""
Registered kernels:
  device='CPU'
  device='GPU'; dtype in [DT_HALF]
  device='GPU'; dtype in [DT_FLOAT]
  device='GPU'; dtype in [DT_DOUBLE]
  device='GPU'; dtype in [DT_INT64]



In [56]:
# testing for integer kernel

tf.reset_default_graph()
with tf.device("/gpu:0"):
    i = tf.Variable(3)
    #i = tf.Variable(3.0) # this works

try:
    tf.Session().run(i.initializer)
except Exception as e:
    print(e)

Cannot assign a device for operation Variable: Could not satisfy explicit device specification '/device:GPU:0' because no supported kernel for GPU devices is available.
Colocation Debug Info:
Colocation group had the following types and supported devices: 
Root Member(assigned_device_name_index_=-1 requested_device_name_='/device:GPU:0' assigned_device_name_='' resource_device_name_='/device:GPU:0' supported_device_types_=[CPU] possible_devices_=[]
VariableV2: CPU 
Assign: CPU 
Identity: GPU CPU 

Colocation members, user-requested devices, and framework assigned devices, if any:
  Variable (VariableV2) /device:GPU:0
  Variable/Assign (Assign) /device:GPU:0
  Variable/read (Identity) /device:GPU:0

Op: VariableV2
Node attrs: shape=[], shared_name="", dtype=DT_INT32, container=""
Registered kernels:
  device='CPU'
  device='GPU'; dtype in [DT_HALF]
  device='GPU'; dtype in [DT_FLOAT]
  device='GPU'; dtype in [DT_DOUBLE]
  device='GPU'; dtype in [DT_INT64]

	 [[node Variable (defined at 

In [57]:
# using soft placement to put this onto a cpu

with tf.device("/gpu:0"):
    t = tf.Variable(3)

config = tf.ConfigProto()
config.allow_soft_placement = True
sess = tf.Session(config=config)
sess.run(i.initializer) # this needs to fall back to /cpu:0

# no error

In [3]:
a = tf.constant(1.0)
b = a + 2.0

with tf.control_dependencies([a,b]):
    x = tf.constant(3.0)
    y = tf.constant(4.0)

z = x + y

# obviously since z depends on x and y, 
# evaluating z also implies waiting for a ans b to be evaluated
# even though it is not explicitly in control_dependecies() block
# since b depends on a we could simplify the preceding code by just creating
# control dependency [b] instead of [a, b]

### Cluster

In [2]:
cluster_spec = tf.train.ClusterSpec({
    "ps" : [
        "127.0.0.1:2221", # /job:ps/task:0
        "127.0.0.1:2222", # /jon:ps/task:1
    ],
    "worker" : [
        "127.0.0.1:2223", # /job:worker/task:0
        "127.0.0.1:2224", # /jon:worker/task:1
        "127.0.0.1:2225"  # /job:worker/task:2
    ]
})

In [3]:
task_ps0 = tf.train.Server(cluster_spec, job_name="ps", task_index=0)
task_ps1 = tf.train.Server(cluster_spec, job_name="ps", task_index=1)
task_worker0 = tf.train.Server(cluster_spec,job_name="worker",task_index=0)
task_worker1 = tf.train.Server(cluster_spec,job_name="worker",task_index=1)
task_worker2 = tf.train.Server(cluster_spec, job_name="worker", task_index=2)

### pinning operation across devices and server

In [7]:
import numpy as np
tf.reset_default_graph()
tf.set_random_seed(42)
np.random.seed(42)

with tf.device("/job:ps"):
    a = tf.Variable(1.0,name="a")

with tf.device("/job:worker"):
    b = a + 2

with tf.device("/job:worker/task:1"):
    c = a + b

In [8]:
with tf.Session("grpc://127.0.0.1:2221") as sess:
    sess.run(a.initializer)
    print(c.eval())

4.0


In [10]:
def reset_graph():
    tf.reset_default_graph()
    tf.set_random_seed(42)
    np.random.seed(42)

reset_graph()

In [12]:
with tf.device(tf.train.replica_device_setter(
    ps_tasks=2,
    ps_device="/job:ps",
    worker_device="/job:worker")):
    v1 = tf.Variable(1.0,name="v1")
    v2 = tf.Variable(2.0,name="v2")
    v3 = tf.Variable(3.0,name="v3")
    s = v1 +v2
    with tf.device("/task:1"):
        p1 = 2 * s
        with tf.device("/cpu:0"):
            p2 = 3 * s

config = tf.ConfigProto()
config.log_device_placement = True

with tf.Session("grpc://127.0.0.1:2221",config=config) as sess:
    v1.initializer.run()

### The old way Readers forqueue

In [13]:
reset_graph()

default1 = tf.constant([5.])
default2 = tf.constant([6])
default3 = tf.constant([7])

dec = tf.decode_csv(tf.constant("1.,,44"),
                    record_defaults=[default1, default2, default3])

with tf.Session() as sess:
    print(sess.run(dec))

[1.0, 6, 44]


In [14]:
reset_graph()

test_csv = open("my_test.csv","w")
test_csv.write("x1, x2, target\n")
test_csv.write("1.,, 0\n")
test_csv.write("4.,5.,1\n")
test_csv.write("7.,8.,0\n")
test_csv.close()

In [15]:
filename_queue = tf.FIFOQueue(capacity=10, dtypes=[tf.string],shapes=[()])
filename = tf.placeholder(tf.string)
enqueue_filename = filename_queue.enqueue([filename])
close_filename_queue = filename_queue.close()

reader = tf.TextLineReader(skip_header_lines=1)
ket, value = reader.read(filename_queue)

x1, x2, target - tf.decode_csv(value, record_defaults=[[-1.],[-1.], [-1.]])
features = tf.stack([x1, x2])

W0605 08:02:25.253579  2792 deprecation.py:323] From <ipython-input-15-fc91fe5c7006>:6: TextLineReader.__init__ (from tensorflow.python.ops.io_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.TextLineDataset`.


NameError: name 'x1' is not defined